In [3]:
#We will scrape the data from the web using mechanicalsoup (https://mechanicalsoup.readthedocs.io/en/stable/) and we will store the data on SQLite (https://www.sqlite.org/index.html)
#We import the necessary packages
!pip install MechanicalSoup
import mechanicalsoup
import sqlite3
import re
import datetime
import time

In [ ]:
#establish the connection and cursor on SQLite
conn = sqlite3.connect('currencies.sqlite')
cur = conn.cursor()

In [ ]:
#Create three tables on SQLite
cur.executescript('''
CREATE TABLE IF NOT EXISTS date (
    id  INTEGER NOT NULL PRIMARY KEY,
    date   DATE UNIQUE);
CREATE TABLE IF NOT EXISTS currencies (
    id  INTEGER NOT NULL PRIMARY KEY,
    curr_ex TEXT UNIQUE);
CREATE TABLE IF NOT EXISTS outcomes (
    curr_ex_id INTEGER, 
    date_id INTEGER, 
    value FLOAT)
''')

In [ ]:
#Create a list with 10 years-dates to scrape the data on
numdays = 3650
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(numdays)]
date_list= [x.strftime('%Y-%m-%d') for x in date_list]

In [ ]:
# Create a for-loop to scrape the data from the site for every date using a Browser with MechanicalSoup and store the data in each SQL table

url= 'https://www.xe.com/currencytables/'
browser= mechanicalsoup.Browser()
page= browser.get(url)
html_page= page.soup
form= html_page('form')[0]
count= 0

for date in date_list[0:2]:
    form.select('input')[0]['value']= date
    try:
        profiles_page= browser.submit(form, url)
    except:
        print('could not retreieve date', date)
        continue
    results= re.findall('...\s/\s...[0-9]*[.][0-9]*', profiles_page.soup.get_text())
    count= count+1
    if count% 50 ==0:
        print('pausing for one second, last date retrieved:', date)
        time.sleep(.5)
    for result in results:
        curr= result[:9]
        value= result[9:]
                
        cur.execute('''
        INSERT OR IGNORE INTO date (date) VALUES (?)''', (date, ))
        cur.execute('''
        SELECT id FROM date WHERE date= ?''', (date, ))
        date_id= cur.fetchone()[0]
        
        cur.execute('''
        INSERT OR IGNORE INTO currencies (curr_ex) VALUES (?)''', (curr, ))
        cur.execute('''
        SELECT id FROM currencies WHERE curr_ex = ?''', (curr, ))
        curr_ex_id= cur.fetchone()[0]
        
        cur.execute('''
        INSERT OR IGNORE INTO outcomes
        (curr_ex_id, date_id, value) 
        VALUES ( ?, ?, ? )''', 
        (curr_ex_id, date_id, value) )
        
    conn.commit()

In [ ]:
# To see the whole picture, you just have to join the tables, like that on SQLlite (example)
cur.execute('''
SELECT currencies.curr_ex AS currencies, date.date, outcomes.value FROM currencies JOIN date JOIN outcomes ON date.id= outcomes.date_id AND currencies.id = outcomes.curr_ex_id WHERE DATE BETWEEN '2020-01-01' AND '2021-01-01'
''')
cur.fetchall()

In [ ]:
#Another example
cur.execute('''
SELECT currencies.curr_ex AS currencies, date.date, outcomes.value FROM currencies JOIN date JOIN outcomes ON date.id= outcomes.date_id AND currencies.id = outcomes.curr_ex_id WHERE DATE BETWEEN '2020-01-01' AND '2021-01-01'
''')

In [ ]:
cur.fetchall()

In [ ]:
cur.close